In [ ]:
#Le code ci-dessous permet d'extraire des données directement de l'API mise à disposition par l'URSSAF.
# https://mon-entreprise.urssaf.fr/simulateurs/indépendant
# Nous utilisons ensuite ces données pour alimenter les tableaux de bord de nos clients et les accompagner 
# dans la gestion de leur entreprise.
# Pour plus d'informations, consultez notre site internet www.needler.fr

In [1]:
import pandas as pd
import numpy as np
import json
import requests

In [2]:
list_rem= np.arange(1000,6000,1000)

In [3]:
df=[]
for i in range(len(list_rem)):
    df_={}
    url = "https://mon-entreprise.urssaf.fr/api/v1/evaluate"
    payload = {
        "expressions": ["dirigeant . indépendant . cotisations et contributions", 
                        "dirigeant . indépendant . cotisations et contributions . cotisations",
                        "dirigeant . indépendant . cotisations et contributions . maladie",
                        "dirigeant . indépendant . cotisations et contributions . retraite de base",
                        "dirigeant . indépendant . cotisations et contributions . retraite complémentaire",
                        "dirigeant . indépendant . cotisations et contributions . indemnités journalières maladie",
                        "dirigeant . indépendant . cotisations et contributions . invalidité et décès",
                        "dirigeant . indépendant . cotisations et contributions . CSG-CRDS",
                        "dirigeant . indépendant . cotisations et contributions . CSG-CRDS . non déductible",
                        "dirigeant . indépendant . cotisations et contributions . formation professionnelle",
                        "impôt . montant"
                       ],
        "situation": {
            "entreprise . imposition": "'IR'",
            "entreprise . activité . nature": "'libérale'",
            "dirigeant . rémunération . totale": str(list_rem[i]),
            "entreprise . catégorie juridique": "''",
            "dirigeant . régime social": "'indépendant'"
        }
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json"
    }
    response = requests.request("POST", url, json=payload, headers=headers).json()
    df_response= pd.json_normalize(response["evaluate"])
    #
    df_['remuneration brute']= list_rem[i]
    df_["total cotisations et contributions"]= df_response['nodeValue'][0]
    df_["cotisations"]= df_response['nodeValue'][1]
    df_["maladie"]= df_response['nodeValue'][2]
    df_["retraite de base"]= df_response['nodeValue'][3]
    df_["retraite complémentaire"]= df_response['nodeValue'][4]
    df_["indemnités journalières maladie"]= df_response['nodeValue'][5]
    df_["invalidité et décès"]= df_response['nodeValue'][6]
    df_["CSG-CRDS"]= df_response['nodeValue'][7]
    df_["CSG-CRDS non déductible"]= df_response['nodeValue'][8]
    df_["formation professionnelle"]= df_response['nodeValue'][9]
    df_["montant impôt"]= df_response['nodeValue'][10]
    df.append(df_)

In [4]:
df_cotisations= pd.DataFrame(df)

In [5]:
df_cotisations

,remuneration brute,total cotisations et contributions,cotisations,maladie,retraite de base,retraite complémentaire,indemnités journalières maladie,invalidité et décès,CSG-CRDS,CSG-CRDS non déductible,formation professionnelle,montant impôt
0,1000,1243,1052,0,898,0,88,66,81,24,110,0
1,2000,1380,1099,0,898,47,88,66,171,51,110,0
2,3000,1532,1160,0,898,108,88,66,262,78,110,0
3,4000,1685,1221,0,898,169,88,66,354,106,110,0
4,5000,1837,1283,0,898,231,88,66,444,133,110,0


In [ ]:
df_cotisations.to_csv('cotisations_2023.csv')